<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/GAN_LSTM_UCIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Load dataset
file_path = "/content/UCIP.csv"  # Ensure this file is uploaded to Colab

df = pd.read_csv(file_path)

# Separate features and target
X = df.drop(columns=['A'])
y = df['A']

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Separate minority and majority class
X_class0 = X_scaled[y == 0]  # Minority class
X_class1 = X_scaled[y == 1]  # Majority class

# Define LSTM-GAN
latent_dim = 10  # Dimensionality of random noise input

def build_generator():
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(1, latent_dim)),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dense(X_class0.shape[1], activation='tanh'),
        RepeatVector(1)
    ])
    return model

def build_discriminator():
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(1, X_class0.shape[1])),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()
discriminator.trainable = False

# GAN Model
gan_input = tf.keras.Input(shape=(1, latent_dim))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training LSTM-GAN
batch_size = 64
epochs = 200

for epoch in range(epochs):
    # Generate fake samples
    noise = np.random.normal(0, 1, (batch_size, 1, latent_dim))
    generated_data = generator.predict(noise)

    # Select real samples from class 0
    idx = np.random.randint(0, X_class0.shape[0], batch_size)
    real_data = X_class0[idx].reshape(batch_size, 1, -1)

    # Labels for training
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train discriminator
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, 1, latent_dim))
    g_loss = gan.train_on_batch(noise, real_labels)

    if epoch % 20 == 0:
        print(f"Epoch {epoch} | D Loss: {d_loss[0]:.4f} | G Loss: {g_loss:.4f}")

# Generate enough samples to balance dataset
num_samples_needed = len(X_class1) - len(X_class0)
noise = np.random.normal(0, 1, (num_samples_needed, 1, latent_dim))
synthetic_data = generator.predict(noise).reshape(num_samples_needed, -1)

# Convert generated data back to original scale
synthetic_data = scaler.inverse_transform(synthetic_data)

# Create balanced dataset
X_balanced = np.vstack((X_scaled, synthetic_data))
y_balanced = np.hstack((y, np.zeros(num_samples_needed)))

# Shuffle dataset
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Train a simple LSTM classifier
classifier = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[1])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = X_train.reshape(-1, 1, X_train.shape[1])
X_test = X_test.reshape(-1, 1, X_test.shape[1])

classifier.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Evaluate performance
y_pred = (classifier.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred)

print(f"Balanced Dataset Accuracy: {accuracy:.4f}")
print(f"Balanced Dataset F1-Score: {f_score:.4f}")

# Save balanced dataset
balanced_df = pd.DataFrame(np.column_stack((y_balanced, X_balanced)), columns=['A'] + list(df.columns[1:]))
balanced_df.to_csv("/content/UCIP_balanced.csv", index=False)

print("Balanced dataset saved as UCIP_balanced.csv")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0 | D Loss: 0.6910 | G Loss: 0.6932
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch 20 | D Loss: 0.6920 | G Loss: 0.6931
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1743/1743 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9843 - loss: 0.1513
Epoch 2/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9858 - loss: 0.0650
Epoch 3/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9864 - loss: 0.0623
Epoch 4/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9863 - loss: 0.0629
Epoch 5/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9860 - loss: 0.0637
Epoch 6/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9862 - loss: 0.0628
Epoch 7/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9859 - loss: 0.0640
Epoch 8/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9863 - loss: 0.0625
Epoch 9/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9860 - loss: 0.0636
Epoch 10/10
1743/1743 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9857 - loss: 0.0648
872/872 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
Balanced Dataset Accuracy: 0.9871
Balanced Dataset F1-Score: 0.9873
Balanced dataset 